In [1]:
import pandas as pd

pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',1000)

In [2]:
df = pd.read_csv(r'C:\Users\comp\Documents\chicago_crime\Cook_County_Assessor_s_Residential_Sales_Data.csv')

In [3]:
df.loc[:, df.columns != 'geom']

,PIN,Property Class,Neighborhood Code,Land Square Feet,Town Code,Type of Residence,Apartments,Wall Material,Roof Material,Rooms,Bedrooms,Basement,Basement Finish,Central Heating,Other Heating,Central Air,Fireplaces,Attic Type,Attic Finish,Half Baths,Design Plan,Cathedral Ceiling,Construction Quality,Renovation,Site Desirability,Garage 1 Size,Garage 1 Material,Garage 1 Attachment,Garage 1 Area,Garage 2 Size,Garage 2 Material,Garage 2 Attachment,Garage 2 Area,Porch,Other Improvements,Building Square Feet,Repair Condition,Multi Code,Number of Commercial Units,Estimate (Land),Estimate (Building),Sale Date,Deed No.,Sale Price,Longitude,Latitude,Census Tract,Total Building Square Feet,Multi Property Indicator,Property Address,Modeling Group,Full Baths,Age,Use,Number of Units,Percent Ownership,Condo Class Factor,Multi-Family Indicator,Large Lot,"Condition, Desirability and Utility",O'Hare Noise,Floodplain,Road Proximity,Condo Strata,Sale Year,Sale Quarter,Sale Half-Year,Sale Quarter of Year,Sale Month of Year,Sale Half of Year,Most Recent Sale,Age Squared,Age Decade,Age Decade Squared,Lot Size Squared,Improvement Size Squared,Pure Market Filter,Garage Indicator,Neigborhood Code (mapping),Square root of lot size,Square root of age,Square root of improvement size,Town and Neighborhood
0,17273100931118,299,40,365398.0,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,9180,14870,06/26/2015,1517742068,53000,-87.622577,41.840229,839200.0,NaN,0,2921 S MICHIGAN AVE,NCHARS,NaN,44,1,679.0,0.00,299.0,NaN,NaN,NaN,0.0,0.0,1.0,NaN,2015,74,37,2,6,1,1.0,1936,4.4,19.36,1.335157e+11,NaN,1,NaN,40,604.481596,6.633250,NaN,7640
1,18013090421010,299,52,19551.0,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,4500,55890,08/03/2018,1821519280,80000,-87.818892,41.810405,819200.0,NaN,0,4431 PRESCOTT AVE,NCHARS,NaN,57,1,24.0,0.04,299.0,NaN,NaN,NaN,0.0,0.0,1.0,1.0,2018,87,44,3,8,2,1.0,3249,5.7,32.49,3.822416e+08,NaN,1,NaN,52,139.824890,7.549834,NaN,2152
2,16094150130000,211,13,4500.0,77,3.0,6.0,2.0,2.0,24.0,12.0,1.0,3.0,2.0,5.0,0.0,0.0,3.0,0,0.0,2.0,0.0,2.0,NaN,2.0,3.0,1.0,2.0,2.0,7.0,0.0,0.0,0.0,3,0.0,7435.0,2.0,2.0,0.0,31500,195710,10/27/2014,1430046150,79000,-87.751110,41.883993,251800.0,NaN,0,5019 W MAYPOLE AVE,MF,6.0,87,2,NaN,NaN,NaN,211.0,NaN,NaN,0.0,0.0,0.0,NaN,2014,72,36,4,10,2,1.0,7569,8.7,75.69,2.025000e+07,55279225.0,1,1.0,13,67.082039,9.327379,86.226446,7713
3,12253140491007,299,80,12148.0,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,8610,87950,03/07/2013,1306650065,90000,-87.817079,41.928035,810800.0,NaN,0,32 CONTI PKWY,NCHARS,NaN,26,1,12.0,0.09,299.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,2013,65,33,1,3,1,1.0,676,2.6,6.76,1.475739e+08,NaN,1,NaN,80,110.217966,5.099020,NaN,2080
4,12111190201042,299,390,48470.0,71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,9750,74600,03/12/2018,1807108202,118000,-87.840859,41.975825,760803.0,NaN,0,5313 N DELPHIA AVE,NCHARS,NaN,45,1,54.0,0.02,299.0,NaN,NaN,NaN,1.0,0.0,0.0,2.0,2018,85,43,1,3,1,1.0,2025,4.5,20.25,2.349341e+09,NaN,1,NaN,390,220.159033,6.708204,NaN,71390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583365,9103010721146,299,50,312580.0,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,10540,48840,04/25/2018,1811501093,89000,-87.858023,42.055981,806104.0,NaN,0,9800 BIANCO TER,NCHARS,NaN,40,1,228.0,0.00,299.

In [4]:
df.dtypes

PIN                                      int64
Property Class                           int64
Neighborhood Code                        int64
Land Square Feet                       float64
Town Code                                int64
Type of Residence                      float64
Apartments                             float64
Wall Material                          float64
Roof Material                          float64
Rooms                                  float64
Bedrooms                               float64
Basement                               float64
Basement Finish                        float64
Central Heating                        float64
Other Heating                          float64
Central Air                            float64
Fireplaces                             float64
Attic Type                             float64
Attic Finish                             int64
Half Baths                             float64
Design Plan                            float64
Cathedral Cei

In [5]:
from geoalchemy2 import Geometry, WKTElement
import  sqlalchemy
from shapely.geometry import Point

pg = open('db_credentials.txt')
db = pg.read()
pg.close()

df.rename(columns = {'Estimate (Land)' : 'Estimate_Land',
                     'Estimate (Building)' : 'Estimate_Building',
                     'Neigborhood Code (mapping)' : 'Neighborhood_Code_mapping'}, inplace = True)

df.dropna(subset = ['Latitude', 'Longitude'], inplace = True)
df['geom'] = df.apply(lambda x : WKTElement(Point(x['Longitude'], x['Latitude']), srid=4326), axis = 1)

engine = sqlalchemy.create_engine(db)
df.to_sql('home_sales', engine, if_exists='replace', chunksize = 10000, index=False, dtype={'geom': Geometry('POINT', srid= 4326)})